# Setup

## Import packages

In [1]:
############################################################################################################################ 
# Get the latest CREST files for each ID within the target folder (dirname)

from pathlib import Path
import json
from sklearn.metrics import pairwise_distances
from scipy.optimize import curve_fit
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from itertools import combinations
from numpy import array, unravel_index, argmin, mean
import random
import numpy as np
from copy import deepcopy
import itertools
from time import time
import neuroglancer
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib
import matplotlib as mpl
import networkx as nx
from webbrowser import open as wb_open
from webbrowser import open_new as wb_open_new
from sqlite3 import connect as sqlite3_connect
from sqlite3 import DatabaseError
from tqdm import tqdm

import sys
sys.path.append('/Users/kperks/Documents/ell-connectome/efish_em/efish_em')

# from eCREST_cli_beta import ecrest, import_settings
from eCREST_cli import ecrest
import AnalysisCode as efish 

In [2]:
vx_sizes = [16, 16, 30]

def get_viewer(backgnd_color = "black"):
    viewer = neuroglancer.Viewer()
    viewer.set_state({})

    location=[17000,17000,1500]

    with viewer.config_state.txn() as s:
        s.show_layer_panel = True ###

    with viewer.txn(overwrite=True) as s:

        dimensions = neuroglancer.CoordinateSpace(
            scales=vx_sizes,# self.vx_sizes['em'],
            units='nm',
            names=['x', 'y', 'z']   )

        s.showSlices = False
        s.dimensions = dimensions
        s.position = array(location)
        s.layout = "3d"
        s.projectionScale = 30000
        s.projection_background_color= matplotlib.colors.cnames[backgnd_color]#"#000000"
        # s.projection_background_color= "#ffffff"
        s.showSlices = False

    with viewer.txn(overwrite=True) as s:
        wb_open(str(viewer))
    
    return viewer

def bkgnd_color(viewer,backgnd_color):
    with viewer.txn(overwrite=True) as s:
        s.projection_background_color= matplotlib.colors.cnames[backgnd_color]

def create_baseseg_layer(viewer, source, lname, visible = False, objectAlpha = 1):
    with viewer.txn(overwrite=True) as s:
        s.layers[lname] = neuroglancer.SegmentationLayer(source = source, segments=[], segment_colors={})
        s.layers[lname].ignoreNullVisibleSet = False
        s.layers[lname].pick = True
        s.layers[lname].selectedAlpha = 0.5 #For 2D
        s.layers[lname].objectAlpha = objectAlpha
        s.layers[lname].visible = visible
                 

def create_em_layer(viewer, source = 'brainmaps://10393113184:ell:roi450um_xyz'):
    with viewer.txn(overwrite=True) as s:
        s.layers['em'] = neuroglancer.ImageLayer(source = source)

def create_mesh_layer(viewer, source, lname):
#     source = = 'precomputed://gs+ngauth+https://ngauth-goog.appspot.com/fish-ell/roi450um_seg32fb16fb_220930'
#precomputed://gs+ngauth+https://ngauth-goog.appspot.com/h01-release/data/20210601/c3
#gs://fish-ell/roi450um_seg32fb16fb_220930
    # lname = 'skel'
    with viewer.txn(overwrite=True) as s:
        s.layers[lname] = neuroglancer.SegmentationLayer(source = source, segments=[], segment_colors={})
        s.layers[lname].ignoreNullVisibleSet = False
        s.layers[lname].pick = True
        s.layers[lname].selectedAlpha = 0.5 #For 2D
        s.layers[lname].meshSilhouetteRendering = 4.2
        s.layers[lname].objectAlpha = 1 #objectAlpha
        s.layers[lname].visible = False #visible

def clear_baseseg_layer(viewer, lname):
    displayed_segs = set([str(x) for x in viewer.state.layers[lname].segments])

    # REMOVE SEGMENTS FROM NGviewer
    remove_segs = displayed_segs

    with viewer.txn(overwrite=True) as s:
        for bs in remove_segs:
            if int(bs) in s.layers[lname].segments:
                s.layers[lname].segments.remove(int(bs))

## Datapath settings

If you save a copy of settings_dict.json (found in the "under construction" directory of eCREST repo) locally somewhere outside the repo (like in your save_dir), then you can use the following code cell to import. This avoids needing to re-type the save_dir and db_path each time you "git pull" updates from the repo to this notebook.

In [3]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = efish.import_settings(path_to_settings_json)

vx_sizes = [16,16,30]

db_cursors = sqlite3_connect(settings_dict['db_path'], check_same_thread=False).cursor()

a = ', '.join(['base_address'])

db_cursors.execute(f'''SELECT {a} FROM addresses_table LIMIT 1''')

[base_seg] = db_cursors.fetchall()[0]


## Plotting style settings

In [4]:
filepath = Path('/Users/kperks/Documents/ell-connectome/efish_em')
filename = 'efish_em.mplstyle'
plt.style.use(filepath/filename)

In [5]:
cell_colors = efish.color_palette('cell')
structure_colors = efish.color_palette('structure')

## Load reconstruction files

In [6]:
dirpath = Path(settings_dict['save_dir'])

nodefiles = efish.get_cell_filepaths(dirpath)

# cell types for all files in directory

## from file

In [7]:
df_type = pd.read_csv(dirpath / 'metadata/df_type_auto_typed.csv')

# df_syn

In [8]:
df_syn = pd.read_csv(dirpath / 'graphs/df_postsyn.csv')
len(df_syn)

20399

In [11]:
# y_adj_col = []
# for i,r in df_syn.iterrows():
#     yoffset = efish.func_planar_curve((r['x'], r['z']), *popt)
#     y_adj = (r['y'] - yoffset)
#     y_adj_col.append(y_adj)

# df_syn.loc[:,'y_adj']=y_adj_col

# # for v in ['x','y','z','y_adj']:
# #     df_syn[v] = df_syn[v]/1000
# df_syn['y_adj'] = df_syn['y_adj']*-1    

## add cell type to df_syn

In [12]:
for i,r in df_syn.iterrows():
    try:
        df_syn.loc[i,'pre_type'] =df_type[df_type['id'].isin([r['pre']])].cell_type.values[0]
        df_syn.loc[i,'post_type']=df_type[df_type['id'].isin([r['post']])].cell_type.values[0]
    except:
        print(r['pre'],r['post'])
        continue

df_syn.loc[:,'post_type'] = [t.lower() for t in df_syn['post_type']]
df_syn.loc[:,'pre_type'] = [t.lower() for t in df_syn['pre_type']]

# neuroglancer viewer

In [13]:
viewer = get_viewer(backgnd_color="white")
create_em_layer(viewer)

In [23]:
lname = 'Gr'
create_baseseg_layer(viewer, base_seg, lname)

In [39]:
mask = df_syn['structure'].isin(['unknown']) & df_syn['post_type'].isin(['lf'])

In [40]:
for i,r in df_syn[mask & df_syn['pre_type'].isin(['mg1'])].iterrows():
    print([int(p/v) for p,v in zip(r[['x','y','z']].values,[16,16,30])])
    # for p in :
    #     print(r)#)]
#  

[23132, 19677, 1754]
[19959, 18926, 555]
[19899, 19032, 410]
[18174, 18865, 531]
[10704, 19190, 1257]
[10311, 19214, 1170]
[15401, 18873, 886]
[15355, 18876, 876]
[15097, 18910, 486]
[14931, 18776, 803]
[23677, 19120, 383]
[23576, 19151, 424]
[23626, 19053, 461]
[23500, 19116, 549]
[23691, 19028, 629]
[23870, 18962, 768]
[26540, 19278, 969]
[26795, 18602, 993]
[26600, 18756, 1052]
[23422, 19713, 2188]
[23436, 19565, 2155]
[23481, 19536, 2151]
[17455, 18976, 1898]
[17469, 18901, 1898]
[17664, 18931, 1975]
[17695, 18905, 2011]
[14517, 18930, 2917]
[14555, 19023, 2906]
[14601, 19020, 2882]
[19987, 18610, 2787]
[19980, 18703, 2767]
[19709, 18820, 2683]
[19488, 18931, 2670]
[19599, 18983, 2662]
[20614, 19134, 1813]
[20858, 18997, 1771]
[21071, 18796, 1781]
[21117, 18748, 1784]
[21234, 18662, 1799]
[21377, 18504, 1825]
[21475, 17929, 1819]
[21582, 17636, 1832]
[19906, 19145, 409]
[19862, 19038, 529]
[19957, 19003, 570]
[20454, 18974, 173]
[20450, 18962, 204]
[20610, 18786, 294]
[20618, 18679

Gr 19262, 16112, 1685
SP 13007, 17916, 1844
SG1 12374, 19480, 676
SG2 8331, 17170, 1458
MG2 7383, 17107, 2484
MG1 21502, 19239, 2070
EAF 17132, 20237, 1973